In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split, LeaveOneOut, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

In [5]:
from sklearn.pipeline import Pipeline


xgBoost_SearchSpace = {
    'booster': ['gblinear'],
    'n_estimators': [225],
    'learning_rate': [.225],
    'reg_lambda': [0, 0.05, .1],
    'reg_alpha': [0, 0.05, .1],
    'eval_metric': ['rmse', 'auc'],
    'feature_selector': ['cyclic', 'random', 'greedy', 'thrifty'],
    'objective': ['reg:squaredlogerror'],
    'updater': ['shotgun', 'coord_descent'],
    'top_k': [0, 1 , 2, 3]            
}

rf_SearchSpace = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Splitting criterion
    'splitter': ['best', 'random'],               # Strategy to choose split at each node
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],      # Maximum depth of the tree
    'min_samples_split': [2, 5, 10, 15, 20],              # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 8, 16],                # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None], # Number of features to consider for the best split
    'max_leaf_nodes': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]        # Maximum number of leaf nodes in the tree
}

svr_SearchSpace = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

lasso_SearchSpace = {
    'alpha': [.1, .125, .175, .2, .15, .18, .17, .165, .185],  # Different values for alpha
    'tol': [.0001],  # Different values for tolerance
    'max_iter': [2000],  # Different values for maximum iterations
    'selection': ['cyclic']
}


scoring = {
    'r2': 'r2',  
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

searchSpace = {
    'hidden_layer_sizes': [(100, 50, 25), (100, 50, 25, 10), (50, 25), (128, 64), (75, 25, 15)],  # Adjusting number of neurons and layers
    'activation': ['relu', 'identity', 'logistic', 'tanh'],  # Activation functions
    'alpha': [0.01, .1, .001, 1],  # L2 regularization parameter
    'learning_rate_init': [0.001, 0.0005, 0.0001, 0.002, 0.005],  # Initial learning rate
    'solver': ['adam', 'lbfgs', 'sgd'],  # Optimization algorithms
    'batch_size': [23],  # Batch size
    'max_iter': [1000],  # Maximum number of iterations
    'random_state': [42]
}


In [6]:


GS = GridSearchCV(estimator=MLPRegressor(),
                  param_grid= searchSpace,
                  scoring = 'neg_mean_absolute_error',
                  refit='neg_mean_absolute_error',
                  cv=5,
                  verbose=4)

In [7]:
GS.fit(X, y)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
[CV 1/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, solver=adam;, score=-18.962 total time=   0.0s
[CV 2/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, solver=adam;, score=-14.820 total time=   0.0s
[CV 3/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, solver=adam;, score=-14.552 total time=   0.0s
[CV 4/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, solver=adam;, score=-4.989 total time=   0.0s
[CV 5/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, sol

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: overflow encountered 

[CV 1/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, solver=sgd;, score=nan total time=   0.2s


C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


[CV 2/5] END activation=relu, alpha=0.01, batch_size=23, hidden_layer_sizes=(100, 50, 25), learning_rate_init=0.001, max_iter=1000, random_state=42, solver=sgd;, score=nan total time=   0.2s


...

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 4/5] END activation=tanh, alpha=1, batch_size=23, hidden_layer_sizes=(75, 25, 15), learning_rate_init=0.005, max_iter=1000, random_state=42, solver=lbfgs;, score=-4.908 total time=   0.2s


C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
756 fits failed out of a total of 6000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
------------------------------

[CV 5/5] END activation=tanh, alpha=1, batch_size=23, hidden_layer_sizes=(75, 25, 15), learning_rate_init=0.005, max_iter=1000, random_state=42, solver=lbfgs;, score=-6.648 total time=   0.2s
[CV 1/5] END activation=tanh, alpha=1, batch_size=23, hidden_layer_sizes=(75, 25, 15), learning_rate_init=0.005, max_iter=1000, random_state=42, solver=sgd;, score=-16.726 total time=   0.0s
[CV 2/5] END activation=tanh, alpha=1, batch_size=23, hidden_layer_sizes=(75, 25, 15), learning_rate_init=0.005, max_iter=1000, random_state=42, solver=sgd;, score=-12.158 total time=   0.0s
[CV 3/5] END activation=tanh, alpha=1, batch_size=23, hidden_layer_sizes=(75, 25, 15), learning_rate_init=0.005, max_iter=1000, random_state=42, solver=sgd;, score=-15.588 total time=   0.0s
[CV 4/5] END activation=tanh, alpha=1, batch_size=23, hidden_layer_sizes=(75, 25, 15), learning_rate_init=0.005, max_iter=1000, random_state=42, solver=sgd;, score=-10.849 total time=   0.0s
[CV 5/5] END activation=tanh, alpha=1, batch

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['relu', 'identity', 'logistic', 'tanh'],
                         'alpha': [0.01, 0.1, 0.001, 1], 'batch_size': [23],
                         'hidden_layer_sizes': [(100, 50, 25),
                                                (100, 50, 25, 10), (50, 25),
                                                (128, 64), (75, 25, 15)],
                         'learning_rate_init': [0.001, 0.0005, 0.0001, 0.002,
                                                0.005],
                         'max_iter': [1000], 'random_state': [42],
                         'solver': ['adam', 'lbfgs', 'sgd']},
             refit='neg_mean_absolute_error', scoring='neg_mean_absolute_error',
             verbose=4)

In [11]:
GS.best_params_

{'activation': 'relu',
 'alpha': 1,
 'batch_size': 23,
 'hidden_layer_sizes': (50, 25),
 'learning_rate_init': 0.005,
 'max_iter': 1000,
 'random_state': 42,
 'solver': 'adam'}

In [12]:
GS.best_score_

-7.467969036746472

In [10]:
GS.best_estimator_

MLPRegressor(alpha=1, batch_size=23, hidden_layer_sizes=(50, 25),
             learning_rate_init=0.005, max_iter=1000, random_state=42)